In [1]:
import scipy as sc
import numpy as np
!pip install gpxpy
import gpxpy

In [2]:
def dtw(s, t, window):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)])
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix

In [3]:
# what do we need for this alg to work:
    # we need the two lists to be (roughly) the same distance
    # how to achieve this: we break up user activity into sizes
#def good_data(segment, user):


In [4]:
def file_to_elevation_arr(fileName):
        gpx_file = open(fileName, 'r')
        gpx = gpxpy.parse(gpx_file)
        elevationOne = []
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    #print('Point at ({0},{1}) -> {2}'.format(point.latitude, point.longitude, point.elevation))
                    elevationOne.append(point.elevation)
        return elevationOne

In [5]:
fileOne = "data/Walt's Strava/activities/4844961125.gpx"
fileTwo  = "data/Walt's Strava/activities/4812864781.gpx"

In [6]:
arrOne = file_to_elevation_arr(fileOne)
arrTwo = file_to_elevation_arr(fileTwo)

In [7]:
print(dtw(arrOne, arrTwo,10))

[[0.00000e+00         inf         inf ...         inf         inf
          inf]
 [        inf 3.00000e-01 6.00000e-01 ...         inf         inf
          inf]
 [        inf 5.00000e-01 5.00000e-01 ...         inf         inf
          inf]
 ...
 [        inf         inf         inf ... 3.73075e+04 3.73102e+04
  3.73129e+04]
 [        inf         inf         inf ...         inf 3.73102e+04
  3.73129e+04]
 [        inf         inf         inf ...         inf         inf
  3.73127e+04]]


In [8]:
import math
!pip install geopy
from geopy import distance

p1 = (43.668613, 40.258916, 0.976)
p2 = (43.658852, 40.250839, 1.475)
flat_distance = distance.distance(p1[:2], p2[:2]).km
print(flat_distance)
euclidian_distance = math.sqrt(flat_distance**2 + (p2[2] - p1[2])**2)
print(euclidian_distance)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.5 MB/s eta 0:00:00
1.265133525952866
1.359986705262199


In [9]:
# points must have elevation data established to call this method
def distanceBetweenPoints(p1, p2):
    flat_distance = distance.distance(p1[:2], p2[:2]).km
    # print(flat_distance)
    euclidian_distance = math.sqrt(flat_distance**2 + (p2[2] - p1[2])**2)
    return euclidian_distance
